In [1]:
import requests
import pandas as pd
import os
from datetime import date, datetime, timedelta

In [18]:
def fetch_data(dno_region_code, start_date, end_date):
    
    headers = {
        "Accept": "application/json"
        }

    r = requests.get(f"https://odegdcpnma.execute-api.eu-west-2.amazonaws.com/development/prices?dno={dno_region_code}&voltage=HV&start={start_date}&end={end_date}",
                    params={}, 
                    headers = headers)

    response = r.json()

    region = {10 : "Eastern England",
            11 : "East Midlands",
            12 : "London",
            13 : "North Wales & Mersey",
            14 : "Midlands",
            15 : "North East",
            16 : "North West",
            17 : "Northern Scotland",
            18 : "Southern Scotland",
            19 : "South East",
            20 : "Southern",
            21 : "South Wales",
            22 : "South Western",
            23 : "Yorkshire"}

    df = pd.json_normalize(response['data']['data'])

    df = df.rename(columns={'Timestamp':'timestamp','Overall':'price'})

    df = df.drop(columns=['unixTimestamp'],axis=1)

    df['region_code'] = dno_region_code

    df['region_name'] = region[dno_region_code]

    df = df[['region_code','region_name','timestamp','price']]

    os.makedirs(f'/Users/gagandeepkundi/Education/data-engineering/data-engineering-projects/energy_data_uk/data_files/{region[dno_region_code]}', exist_ok=True)

    return df.to_csv(f'/Users/gagandeepkundi/Education/data-engineering/data-engineering-projects/energy_data_uk/data_files/{region[dno_region_code]}/{start_date}-{region[dno_region_code]}.csv',index=False)

In [12]:
def get_first_date_of_current_month(year, month):
    first_date = datetime(year, month, 1)
    return first_date.strftime("%d-%m-%Y")

def get_last_date_of_month(year, month):
    if month == 12:
        last_date = datetime(year, month, 31)
    else:
        last_date = datetime(year, month + 1, 1) + timedelta(days=-1)
    
    return last_date.strftime("%d-%m-%Y")

In [20]:
for j in range(1,13):
    start_date = get_first_date_of_current_month(2022,j)
    end_date = get_last_date_of_month(2022,j)
    for region in range(10, 24):
        fetch_data(region, start_date=start_date, end_date=end_date)